### Week 3 - Segmenting and Clustering Neighborhoods in Toronto
### Question 2 - Get Coordinates

#### Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#you may need to install bs4 on your system, uncomment below line to do so
#pip install beautifulsoup4
from bs4 import BeautifulSoup
print('Libraries imported.')

Libraries imported.


#### Import CSV (since table already formatted)

In [2]:
df =pd.read_csv('Toronto Data.csv')
df.head()

,Unnamed: 0,PostalCode,Borough,Neighborhood
0,0,M1B,Scarborough,"Malvern, Rouge"
1,1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,3,M1G,Scarborough,Woburn
4,4,M1H,Scarborough,Cedarbrae


#### Drop "Index" Column from CSV

In [4]:
df.drop('Unnamed: 0',axis=1, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Get the Coordinates

In [8]:
pip install geocoder

     |████████████████████████████████| 102kB 17.5MB/s ta 0:00:01
     |████████████████████████████████| 92kB 6.3MB/s eta 0:00:011
     |████████████████████████████████| 829kB 15.0MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [11]:
import geocoder

#### Create a function to make sure this is working

In [17]:

def get_lat_long(postal_code, max_iterations, debug):
    lat_long = None
    success=False
    for i in range(max_iterations):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_long = g.latlng
        if debug:
            print('Attempt #: {}, Coordinates: {}'.format(i+1, lat_long))
        if lat_long is not None:
            success=True
            break
    if success:
        latitude = lat_long[0]
        longitude = lat_long[1]
        return latitude, longitude
    else:
        return 'Failed to get coordinates'

#### Try to get coordinates

In [18]:
get_lat_long('M1C',10,True)

Attempt #: 1, Coordinates: None
Attempt #: 2, Coordinates: None
Attempt #: 3, Coordinates: None
Attempt #: 4, Coordinates: None
Attempt #: 5, Coordinates: None
Attempt #: 6, Coordinates: None
Attempt #: 7, Coordinates: None
Attempt #: 8, Coordinates: None
Attempt #: 9, Coordinates: None
Attempt #: 10, Coordinates: None


'Failed to get coordinates'

In [20]:
get_lat_long('M1C',300,False)

'Failed to get coordinates'

#### Try to get any response

In [21]:
g = geocoder.google('Washington, D.C.')
print(g.latlng)

None


#### Since that was a giant waste of time we will use the csv

#### Import CSV and save it as a DataFrame

In [22]:
coords = pd.read_csv('https://cocl.us/Geospatial_data')
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Make sure the common column in both DataFrames has the same label

In [23]:
coords.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df.rename(columns={'Postcode': 'PostalCode'}, inplace=True)

#### Merge DataFrames

In [24]:
df_Merged = pd.merge(df, coords, on='PostalCode')
df_Merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Save to CSV for future use

In [26]:
df_Merged.to_csv('Toronto Data Final.csv')